In [1]:
import pandas as pd
import sys
import os

In [2]:
path = "/Users/irbraun/phenologs-with-oats/outputs/stacked_10_14_2020_h08m41s58_6879_rev"


results = pd.read_csv(os.path.join(path, "stacked_full_table_with_all_metrics.csv"))
results

,method,species,objective,curated,hyperparameters,group,order,mean_1,mean_0,n_1,n_0,ks,ks_pval,auc,auc_baseline,f1_max_baseline,f1_max,f5_max,f2_max
0,baseline,inter,orthologs,False,identity,nlp,0,1.000000,1.000000,59.0,217208.0,0.000000,1.000000e+00,0.500136,0.000272,0.000543,0.000543,0.000339,0.001356
1,baseline,intra,known,True,identity,nlp,0,0.999814,0.999902,32205.0,1100731.0,0.000088,1.000000e+00,0.040619,0.028426,0.055281,0.055281,0.035282,0.127620
2,baseline,intra,subsets,True,identity,nlp,0,0.999282,0.999994,161519.0,994441.0,0.000712,9.999998e-01,0.545582,0.139727,0.245194,0.245194,0.168764,0.448157
3,baseline,inter,pathways,True,identity,nlp,0,1.000000,1.000000,1147.0,45376.0,0.000000,1.000000e+00,0.512327,0.024654,0.048123,0.048123,0.030629,0.112207
4,baseline,both,pathways,True,identity,nlp,0,0.999313,0.999936,8739.0,266914.0,0.000623,1.000000e+00,0.146619,0.031703,0.061457,0.061457,0.039317,0.140675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,biobert,intra,pathways,False,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.055245,0.065445,23846.0,734697.0,0.157707,0.000000e+00,0.105938,0.031437,0.060957,0.114167,0.096204,0.167349
808,biobert,intra,predicted,False,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.062154,0.068270,49750.0,5009867.0,0.101522,0.000000e+00,0.050202,0.009833,0.019474,0.031195,0.022362,0.059260
809,biobert,inter,orthologs,True,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.047412,0.061990,22.0,39335.0,0.371256,3.141892e-03,0.047447,0.000559,0.001117,0.011994,0.007724,0.027285
810,biobert,both,pathways,True,tokenization_pubmed_pmc_layers_4_summed,nlp,2033,0.049195,0.060580,8739.0,266914.0,0.188324,3.057105e-263,0.117596,0.031703,0.061457,0.123331,0.100348,0.184415


In [3]:
results.dtypes

method              object
species             object
objective           object
curated               bool
hyperparameters     object
group               object
order                int64
mean_1             float64
mean_0             float64
n_1                float64
n_0                float64
ks                 float64
ks_pval            float64
auc                float64
auc_baseline       float64
f1_max_baseline    float64
f1_max             float64
f5_max             float64
f2_max             float64
dtype: object

In [4]:
variable_combinations = [
    (True,"subsets","intra"),
    (True,"known","intra"),
    (True,"predicted","intra"),
    (True,"pathways","intra"),
    (True,"pathways","inter"),
    (True,"pathways","both"),
    (True,"orthologs","inter"),
    (False,"subsets","intra"),
    (False,"known","intra"),
    (False,"predicted","intra"),
    (False,"pathways","intra"),
    (False,"pathways","inter"),
    (False,"pathways","both"),
    (False,"orthologs","inter"),
]


In [5]:
# Make another version of the table that is more useful for looking at one particular metric or value.
metrics_of_interest = ["f2_max"]
for metric_of_interest in metrics_of_interest:
    reshaped_results = results[["method","hyperparameters","order","group"]].drop_duplicates()
    for (c,q,s) in variable_combinations:

        # Construct a column name that corresponds to a particular subset.
        c_str = {True:"curated",False:"all"}[c]
        q_str = str(q).lower()
        s_str = str(s).lower()
        col_name =  "{}_{}_{}".format(s_str, c_str, q_str)

        # Pull data out of the the full metrics dataframe.
        reshaped_results[col_name] = reshaped_results["order"].map(lambda x: results.loc[(results["order"]==x) & (results["curated"]==c) & (results["objective"]==q.lower()) & (results["species"]==s.lower()), metric_of_interest])
        reshaped_results[col_name] = reshaped_results[col_name].map(lambda x: None if len(x)==0 else x.values[0])
    
    # Remove columns that have all NA values, these are for questions that weren't applicable to these metrics.
    #reshaped_results.dropna(axis="columns", how="all", inplace=True)
    reshaped_results.to_csv(os.path.join(path, "stacked_{}.csv").format(metric_of_interest), index=False)
